In [1]:
from vierlinden.config import model_output_path
from vierlinden.data.loader import VierlindenDataProcessor
from vierlinden.model.model import NHiTSTrainingWrapper, NHiTSPredictionWrapper

seed = 42

context_length = 50
prediction_length = 10
batch_size = 32
num_workers = 18

def main(arg : str):
    dp = VierlindenDataProcessor()
    df = dp.load_processed_data()
    df = dp.prepare_for_target(df, arg)
    
    training_df, test_df = dp.split_data(df)
    
    nhits_wrapper = NHiTSTrainingWrapper(training_df, 
                                         target_col = arg, 
                                         context_length = context_length, 
                                         prediction_length = prediction_length,
                                         batch_size = batch_size, 
                                         num_workers = num_workers)
    optimal_lr = nhits_wrapper.find_optimal_learningrate(seed = seed)
    
    print(f"Optimal learning rate for {arg}: {optimal_lr}")
    
    best_model = nhits_wrapper.train(optimal_lr, seed = seed, max_epochs=2)
    
    return nhits_wrapper, test_df

/storage/.venv/lib/python3.10/site-packages/pytorch_forecasting/models/base_model.py:30: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
dp = VierlindenDataProcessor()
df = dp.load_processed_data()
df = dp.prepare_for_target(df, 'Kaiserstr_outflow [l/s]')
dp.plot_target_col('Kaiserstr_outflow [l/s]')

In [ ]:
nhits_wrapper, test_df = main('Kaiserstr_outflow [l/s]')

In [ ]:
model_output_path

In [2]:
dp = VierlindenDataProcessor()
df = dp.load_processed_data()
df = dp.prepare_for_target(df, 'Kaiserstr_outflow [l/s]')

training_df, test_df = dp.split_data(df)
test_df.shape

INFO:vierlinden.data.loader:Data loaded successfully from /storage/projects/RIWWER/data/Vierlinden
INFO:vierlinden.data.loader:Sensor and target data merged successfully.
INFO:vierlinden.data.loader:NaN values processed successfully.
INFO:vierlinden.data.loader:Data loaded and processed successfully.


(868, 24)

In [ ]:
model = nhits_wrapper.best_model

In [3]:
model = NHiTSTrainingWrapper.load_trained_model(model_output_path + "/"  + 'test.pt')

In [4]:
from vierlinden.model.model import NHiTSPredictionWrapper
prediction_wrapper = NHiTSPredictionWrapper(model, context_length=context_length, prediction_length=prediction_length)
something = prediction_wrapper.predict(test_df, 'Kaiserstr_outflow [l/s]')

9


/storage/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


In [5]:
import pandas as pd
import numpy as np

output_df = pd.DataFrame(something.output.prediction.detach().cpu().numpy()[:,:,0])
output_df["decoder_length"] = something.decoder_lengths.detach().cpu()
output_df = output_df.join(something.index)
display(output_df)

# Why does pytorch forecasting not predict for the last 11 timeindices?
# print(np.array(something.index["time_idx"]))
# print(something.x["decoder_time_idx"].detach().cpu().numpy())
# np.array_equal(np.array(something.index["time_idx"]), something.x["decoder_time_idx"].detach().cpu().numpy())

# something.x is probably not worth exposing / returning
# something.y is probably not worth exposing / returning

# The dataframe shows for every time index in time_idx column the values for the next 10 time indices -> That means value in column 9 for time idx 55 corresponds to the value for time idx 65 (0 corresponds to 56)

,0,1,2,3,4,5,6,7,8,9,decoder_length,time_idx,series
0,-1.886773e-08,-5.521628e-08,-3.555689e-08,-3.540658e-08,-3.113318e-08,-4.963855e-08,-4.343823e-08,-5.617491e-08,-3.855793e-08,-5.542795e-08,10,51,0
1,-1.682443e-08,-3.684833e-08,-2.303030e-08,-2.422456e-08,-1.577111e-08,-2.903560e-08,-2.805623e-08,-3.645483e-08,-2.459658e-08,-3.142153e-08,10,52,0
2,-2.880990e-08,-5.954436e-08,-4.045778e-08,-5.864715e-08,-5.192981e-08,-5.145825e-08,-3.721264e-08,-4.882086e-08,-4.243840e-08,-6.180337e-08,10,53,0
3,-5.830474e-08,-8.270424e-08,-6.291569e-08,-8.291109e-08,-6.149386e-08,-6.071738e-08,-4.949933e-08,-7.510833e-08,-6.516339e-08,-8.930824e-08,10,54,0
4,-7.040162e-08,-9.388915e-08,-7.216354e-08,-8.012906e-08,-8.196793e-08,-8.893434e-08,-6.072423e-08,-8.232725e-08,-6.845320e-08,-9.646287e-08,10,55,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,3.087686e+00,1.418103e+00,1.968570e+00,1.166096e+00,2.055196e+00,2.096983e+00,3.062217e+00,2.560739e+00,3.360461e+00,2.419349e+00,10,864,0
814,7.702017e-01,-7.522506e-01,4.382892e-01,-5.553329e-02,3.637590e-01,5.837681e-01,1.007415e+00,8.957375e-01,1.066373e+00,3.653450e-01,10,865,0
815,-9.512991e-01,-1.762932e+00,-1.067788e+00,-1.083850e+00,-2.630400e-01,-5.990751e-01,-5.427745e-01,-7.708578e-01,-3.561515e-01,-5.113930e-01,10,866,0
816,3.265783e-01,-1.559941e-01,4.288068e-02,3.979130e-01,5.246159e-01,7.522476e-02,6.467418e-01,-6.038851e-02,3.754032e-01,5.889149e-01,10,867,0


In [ ]:
import matplotlib.pyplot as plt

plt.plot(something.x["encoder_target"].detach().cpu().numpy())

In [ ]:
for time_idx in range(0, 817, 60):
    fig = model.plot_prediction(something.x, something.output, idx=time_idx, add_loss_to_title=True)

In [ ]:
from matplotlib import pyplot as plt

plt.plot(df_preds[3], label='predictions')
plt.plot(test_df['Kaiserstr_outflow [l/s]'].values, label='actual')
plt.legend()

In [ ]:
print(nhits_wrapper.metrics_callback.metrics["train_loss"])
print(nhits_wrapper.metrics_callback.metrics["val_loss"])

nhits_wrapper.plot_training_result()

In [ ]:
nhits_wrapper.save_trained_model(model_output_path + "/"  + 'test.pt')

In [ ]:
model = NHiTSTrainingWrapper.load_trained_model(model_output_path + "/"  + 'NHiTS_Kaiserstr_model.pt')

In [ ]:
print(model)